# 02 – GDELT 2.1 Feature Engineering

This notebook constructs a country-month level research dataset 
from event-level GDELT 2.1 exports stored in gdelt.db.

In [1]:
import duckdb

con = duckdb.connect("../data/intermediate/gdelt.db")

In [14]:
con.execute("SHOW TABLES").fetchdf()

,name
0,country_month_features
1,exports
2,incoming_country_month
3,outgoing_country_month


In [3]:
con.execute("""
CREATE OR REPLACE TABLE outgoing_country_month AS
SELECT
    Actor1CountryCode AS country,
    Year,
    MonthYear,
    COUNT(*) AS total_events_out,
    SUM(CASE WHEN QuadClass = 4 THEN 1 ELSE 0 END) AS out_conflict,
    AVG(AvgTone) AS avg_tone_out,
    AVG(GoldsteinScale) AS avg_goldstein_out,
    STDDEV(AvgTone) AS tone_volatility_out,
    AVG(NumMentions) AS avg_mentions_out,
    AVG(NumArticles) AS avg_articles_out
FROM exports
WHERE Actor1CountryCode IS NOT NULL
GROUP BY country, Year, MonthYear
""")

In [4]:
con.execute("SELECT * FROM outgoing_country_month LIMIT 5").fetchdf()

,country,Year,MonthYear,total_events_out,out_conflict,avg_tone_out,avg_goldstein_out,tone_volatility_out,avg_mentions_out,avg_articles_out
0,BHS,2025,202512,1653,242.0,-1.741722,0.588203,5.135921,3.597096,3.573503
1,BRA,2025,202512,5790,640.0,-1.400523,1.122418,3.787093,3.993092,3.917098
2,CMR,2025,202512,835,144.0,-2.575544,0.238683,5.454982,4.276647,4.201198
3,ESP,2025,202512,8674,1194.0,-1.124080,0.832938,4.457353,4.115518,4.037929
4,FRA,2025,202512,24433,2439.0,-1.179080,1.152274,3.987349,3.871813,3.801948


In [5]:
con.execute("""
CREATE OR REPLACE TABLE incoming_country_month AS
SELECT
    Actor2CountryCode AS country,
    Year,
    MonthYear,
    COUNT(*) AS total_events_in,
    SUM(CASE WHEN QuadClass = 4 THEN 1 ELSE 0 END) AS in_conflict
FROM exports
WHERE Actor2CountryCode IS NOT NULL
GROUP BY country, Year, MonthYear
""")

In [6]:
con.execute("SELECT * FROM incoming_country_month LIMIT 5").fetchdf()

,country,Year,MonthYear,total_events_in,in_conflict
0,AUS,2025,202512,38563,7817.0
1,AUT,2025,202512,2027,185.0
2,TUN,2025,202512,1008,146.0
3,URY,2025,202512,443,11.0
4,ARM,2025,202512,3819,259.0


In [7]:
con.execute("""
CREATE OR REPLACE TABLE country_month_features AS
SELECT
    COALESCE(o.country, i.country) AS country,
    COALESCE(o.Year, i.Year) AS Year,
    COALESCE(o.MonthYear, i.MonthYear) AS MonthYear,

    COALESCE(o.total_events_out,0) AS total_events_out,
    COALESCE(i.total_events_in,0) AS total_events_in,

    COALESCE(o.out_conflict,0) AS out_conflict,
    COALESCE(i.in_conflict,0) AS in_conflict,

    (COALESCE(o.out_conflict,0) - COALESCE(i.in_conflict,0)) AS net_conflict,

    o.avg_tone_out,
    o.avg_goldstein_out,
    o.tone_volatility_out,
    o.avg_mentions_out,
    o.avg_articles_out

FROM outgoing_country_month o
FULL OUTER JOIN incoming_country_month i
ON o.country = i.country
AND o.Year = i.Year
AND o.MonthYear = i.MonthYear
""")

In [8]:
con.execute("""
CREATE OR REPLACE TABLE country_month_features AS
SELECT *,
    CASE 
        WHEN total_events_out > 0 
        THEN out_conflict::DOUBLE / total_events_out
        ELSE 0 
    END AS conflict_ratio_out,

    CASE 
        WHEN total_events_in > 0 
        THEN in_conflict::DOUBLE / total_events_in
        ELSE 0 
    END AS conflict_ratio_in

FROM country_month_features
""")

In [9]:
con.execute("""
SELECT *
FROM country_month_features
ORDER BY ABS(net_conflict) DESC
LIMIT 10
""").fetchdf()

,country,Year,MonthYear,total_events_out,total_events_in,out_conflict,in_conflict,net_conflict,avg_tone_out,avg_goldstein_out,tone_volatility_out,avg_mentions_out,avg_articles_out,conflict_ratio_out,conflict_ratio_in
0,USA,2026,202601,596733,431823,96737.0,66192.0,30545.0,-2.401044,0.218075,4.049212,3.990259,3.880871,0.162111,0.153285
1,USA,2025,202512,503121,354521,78643.0,56271.0,22372.0,-2.149044,0.439252,4.211073,4.014545,3.897643,0.156310,0.158724
2,USA,2026,202602,261800,186511,37056.0,25706.0,11350.0,-2.070278,0.607946,4.037369,3.981902,3.863717,0.141543,0.137826
3,ISR,2025,202512,57224,52177,13246.0,8891.0,4355.0,-4.128949,-0.793606,3.934880,3.381309,3.328219,0.231476,0.170401
4,ISR,2026,202601,52663,46956,11568.0,8243.0,3325.0,-3.723636,-0.680875,4.037914,3.379147,3.322598,0.219661,0.175547
5,GBR,2026,202601,96042,68762,10811.0,7845.0,2966.0,-1.405640,0.803020,3.909868,4.342902,4.218686,0.112565,0.114089
6,VEN,2026,202601,51525,55732,8941.0,11482.0,-2541.0,-3.160492,-0.218987,2.897942,3.310898,3.257700,0.173527,0.206022
7,GBR,2025,202512,81922,59268,10051.0,7533.0,2518.0,-1.421374,0.788407,4.380891,4.364713,4.231452,0.122690,0.127101
8,IND,2026,202601,40754,26315,5507.0,3151.0,2356.0,-1.535966,0.258323,4.657243,4.177504,4.093709,0.135128,0.119742
9,IND,2025,202512,41952,27565,5849.0,3539.0,2310.0,-1.773401,0.270612,4.787886,4.110245,4.027984,0.139421,0.128387
